In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == 

In [ ]:
import pandas as pd

balanced_tweets = pd.read_csv('/content/Balanced_Tweets.csv')

In [ ]:
balanced_tweets.columns

Index(['Unnamed: 0', 'Sentiment', 'CleanTweet'], dtype='object')

In [ ]:
from datasets import Dataset
from sklearn.model_selection import train_test_split
SEED = 42


train_texts, test_texts, train_labels, test_labels = train_test_split(
    balanced_tweets['CleanTweet'], balanced_tweets['Sentiment'], test_size=0.8, random_state=SEED, stratify=balanced_tweets['Sentiment']
)

train_dataset = Dataset.from_dict({'CleanTweet': train_texts, 'Sentiment': train_labels})
test_dataset = Dataset.from_dict({'CleanTweet': test_texts, 'Sentiment': test_labels})

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def tokenize_function(examples):
    """
    Μετατρέπει τα κείμενα σε tokens χρησιμοποιώντας τον tokenizer του προεκπαιδευμένου μοντέλου.

    Κάθε κείμενο μετατρέπεται σε ακολουθία tokens με padding μέχρι το μέγιστο μήκος (max_length)
    και με περικοπή (truncation) ώστε να μην ξεπερνά το όριο των 512 tokens.

    :param examples: Λεξικό που περιέχει το κείμενο υπό το κλειδί 'text'.
    :return: Λεξικό με τα tokenized αποτελέσματα.
    """
    return tokenizer(
        examples['CleanTweet'],
        padding="max_length",  # Προσθήκη padding ώστε όλα τα sequences να έχουν το ίδιο μήκος.
        truncation=True,       # Ενεργοποίηση του truncation για να περιοριστούν τα sequences στο max_length.
        max_length=512,        # Ορισμός μέγιστου μήκους ακολουθίας.
    )

# Εφαρμογή της συνάρτησης tokenization στο σύνολο εκπαίδευσης και δοκιμής σε παρτίδες.
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Μετονομασία της στήλης "variety" σε "labels", όπως απαιτείται από το μοντέλο της Hugging Face.
train_dataset = train_dataset.rename_column("Sentiment", "labels")
test_dataset = test_dataset.rename_column("Sentiment", "labels")

# Ορισμός του format των δεδομένων σε PyTorch tensors για να μπορούν να χρησιμοποιηθούν από το μοντέλο.
train_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Ορισμός των παραμέτρων εκπαίδευσης (TrainingArguments) για την προσαρμογή του μοντέλου.
training_args = TrainingArguments(
    output_dir="./results",                # Ο φάκελος όπου θα αποθηκεύονται τα αποτελέσματα και τα checkpoints του μοντέλου.
    evaluation_strategy="epoch",           # Αξιολόγηση του μοντέλου στο τέλος κάθε εποχής.
    learning_rate=2e-5,                    # Ρυθμός μάθησης για την εκπαίδευση.
    per_device_train_batch_size=8,         # Μέγεθος batch ανά συσκευή κατά την εκπαίδευση.
    num_train_epochs=10,                   # Συνολικός αριθμός εποχών εκπαίδευσης.
    weight_decay=0.01,                     # Συντελεστής αποσύνθεσης βάρους για την κανονικοποίηση.
    logging_dir="./logs",                  # Φάκελος όπου θα αποθηκεύονται τα logs.
    logging_steps=10,                      # Ο αριθμός των βημάτων μετά από τα οποία γίνεται logging.
    save_strategy="epoch",                 # Αποθήκευση του μοντέλου στο τέλος κάθε εποχής.
    load_best_model_at_end=True,           # Φόρτωση του καλύτερου μοντέλου στο τέλος της εκπαίδευσης.
    metric_for_best_model="f1",            # Ορισμός του μετρικού (F1) για την αξιολόγηση του καλύτερου μοντέλου.
    greater_is_better=True                 # Ορισμός ότι μεγαλύτερες τιμές του F1 είναι καλύτερες.
)

# Ορισμός του callback για early stopping, ώστε να σταματήσει η εκπαίδευση εάν δεν υπάρχει βελτίωση.
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,             # Αριθμός εποχών χωρίς βελτίωση πριν τη διακοπή της εκπαίδευσης.
    early_stopping_threshold=0.001         # Κατώφλι βελτίωσης που πρέπει να επιτευχθεί για να θεωρηθεί ότι υπάρχει πρόοδος.
)

def compute_metrics(eval_pred):
    """
    Συνάρτηση για τον υπολογισμό των μετρικών αξιολόγησης του μοντέλου.

    :param eval_pred: Δομή που περιέχει τα logits (προβλέψεις) και τις αληθινές ετικέτες.
    :return: Λεξικό με τις μετρικές ακρίβειας (accuracy), F1-score, precision και recall.
    """
    logits, labels = eval_pred
    # Επιλογή της ετικέτας με τη μέγιστη τιμή πιθανότητας για κάθε δείγμα.
    predictions = logits.argmax(axis=-1)

    # Υπολογισμός των μετρικών precision, recall, f1 με χρήση του μέσου όρου "binary" (δυαδική ταξινόμηση).
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro")

    # Υπολογισμός της ακρίβειας.
    acc = accuracy_score(labels, predictions)

    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Δημιουργία του Trainer, ο οποίος θα χειριστεί την εκπαίδευση και την αξιολόγηση του μοντέλου.
trainer = Trainer(
    model=model,                           # Το μοντέλο που θα εκπαιδευτεί.
    args=training_args,                    # Οι ορισμένες παράμετροι εκπαίδευσης.
    train_dataset=train_dataset,           # Το σύνολο δεδομένων εκπαίδευσης.
    eval_dataset=test_dataset,             # Το σύνολο δεδομένων αξιολόγησης (δοκιμής).
    compute_metrics=compute_metrics,       # Συνάρτηση υπολογισμού των μετρικών αξιολόγησης.
    callbacks=[early_stopping]             # Callback για το early stopping.
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.843100,0.826786,0.656944,0.656944,0.656944,0.656944
2,0.659800,0.722320,0.710972,0.710972,0.710972,0.710972
3,0.377100,0.972768,0.698333,0.698333,0.698333,0.698333
4,0.152900,1.261762,0.709306,0.709306,0.709306,0.709306


TrainOutput(global_step=900, training_loss=0.5856535345150364, metrics={'train_runtime': 1646.535, 'train_samples_per_second': 10.932, 'train_steps_per_second': 1.367, 'total_flos': 1894416607641600.0, 'train_loss': 0.5856535345150364, 'epoch': 4.0})

In [ ]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.7223196625709534, 'eval_accuracy': 0.7109722222222222, 'eval_f1': 0.7109722222222222, 'eval_precision': 0.7109722222222222, 'eval_recall': 0.7109722222222222, 'eval_runtime': 216.168, 'eval_samples_per_second': 33.307, 'eval_steps_per_second': 4.163, 'epoch': 4.0}


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Πρόβλεψη στο test set χρησιμοποιώντας τον εκπαιδευμένο trainer.
predictions = trainer.predict(test_dataset)

# Εξαγωγή των αληθινών ετικετών (labels) από το test_dataset.
y_true = np.array([example['labels'] for example in test_dataset])

# Εξαγωγή των προβλεπόμενων ετικετών: επιλέγουμε την κλάση με τη μεγαλύτερη τιμή πιθανοτήτων για κάθε δείγμα.
y_pred = np.argmax(predictions.predictions, axis=1)


# Εκτύπωση του Classification Report για την αξιολόγηση της απόδοσης του μοντέλου.
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

Classification Report:
              precision    recall  f1-score   support

    Negative       0.69      0.79      0.74      2400
     Neutral       0.74      0.67      0.70      2400
    Positive       0.72      0.67      0.69      2400

    accuracy                           0.71      7200
   macro avg       0.71      0.71      0.71      7200
weighted avg       0.71      0.71      0.71      7200



## ELMO

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from tqdm import tqdm

# Load ELMo from TF-Hub
elmo = hub.load("https://tfhub.dev/google/elmo/3")

def elmo_embed(texts):
    """
    Get ELMo embeddings for a list of texts.
    Returns a matrix of shape (num_texts, 1024)
    """
    embeddings = []
    batch_size = 32

    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        result = elmo.signatures["default"](tf.constant(batch))["default"]
        embeddings.append(result.numpy())

    return np.vstack(embeddings)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

balanced_tweets = pd.read_csv('/content/Balanced_Tweets.csv')

SEED = 42

train_texts, test_texts, train_labels, test_labels = train_test_split(
    balanced_tweets['CleanTweet'],
    balanced_tweets['Sentiment'],
    test_size=0.8,
    random_state=SEED,
    stratify=balanced_tweets['Sentiment']
)

In [ ]:
# Convert to lists
train_texts = train_texts.tolist()
test_texts = test_texts.tolist()

# Compute ELMo embeddings
X_train_elmo = elmo_embed(train_texts)
X_test_elmo = elmo_embed(test_texts)

100%|██████████| 225/225 [00:49<00:00,  4.53it/s]


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

clf = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=SEED)
clf.fit(X_train_elmo, train_labels)

# Predict
y_pred = clf.predict(X_test_elmo)

In [ ]:
print("Classification Report:")
print(classification_report(test_labels, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

Classification Report:
              precision    recall  f1-score   support

    Negative       0.62      0.59      0.60      2400
     Neutral       0.62      0.61      0.61      2400
    Positive       0.58      0.61      0.59      2400

    accuracy                           0.60      7200
   macro avg       0.60      0.60      0.60      7200
weighted avg       0.60      0.60      0.60      7200



## XLNet

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import numpy as np
SEED = 42


balanced_tweets = pd.read_csv('/content/Balanced_Tweets.csv')
# balanced_tweets = balanced_tweets.sample(frac=0.5, weights='Sentiment', random_state=SEED)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    balanced_tweets['CleanTweet'], balanced_tweets['Sentiment'], test_size=0.8, random_state=SEED, stratify=balanced_tweets['Sentiment']
)

train_dataset = Dataset.from_dict({'CleanTweet': train_texts, 'Sentiment': train_labels})
test_dataset = Dataset.from_dict({'CleanTweet': test_texts, 'Sentiment': test_labels})

In [ ]:
model_name = "xlnet-base-cased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=10)

def tokenize_function(examples):
    """
    Μετατρέπει τα κείμενα σε tokens χρησιμοποιώντας τον tokenizer του προεκπαιδευμένου μοντέλου.

    Κάθε κείμενο μετατρέπεται σε ακολουθία tokens με padding μέχρι το μέγιστο μήκος (max_length)
    και με περικοπή (truncation) ώστε να μην ξεπερνά το όριο των 512 tokens.

    :param examples: Λεξικό που περιέχει το κείμενο υπό το κλειδί 'text'.
    :return: Λεξικό με τα tokenized αποτελέσματα.
    """
    return tokenizer(
        examples['CleanTweet'],
        padding="max_length",  # Προσθήκη padding ώστε όλα τα sequences να έχουν το ίδιο μήκος.
        truncation=True,       # Ενεργοποίηση του truncation για να περιοριστούν τα sequences στο max_length.
        max_length=512,        # Ορισμός μέγιστου μήκους ακολουθίας.
    )

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Εφαρμογή της συνάρτησης tokenization στο σύνολο εκπαίδευσης και δοκιμής σε παρτίδες.
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Μετονομασία της στήλης "variety" σε "labels", όπως απαιτείται από το μοντέλο της Hugging Face.
train_dataset = train_dataset.rename_column("Sentiment", "labels")
test_dataset = test_dataset.rename_column("Sentiment", "labels")

# Ορισμός του format των δεδομένων σε PyTorch tensors για να μπορούν να χρησιμοποιηθούν από το μοντέλο.
train_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
    output_dir="./xlnet-results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./xlnet-logs",
    logging_steps=10,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=42
)

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,             # Αριθμός εποχών χωρίς βελτίωση πριν τη διακοπή της εκπαίδευσης.
    early_stopping_threshold=0.001         # Κατώφλι βελτίωσης που πρέπει να επιτευχθεί για να θεωρηθεί ότι υπάρχει πρόοδος.
)

def compute_metrics(eval_pred):
    """
    Συνάρτηση για τον υπολογισμό των μετρικών αξιολόγησης του μοντέλου.

    :param eval_pred: Δομή που περιέχει τα logits (προβλέψεις) και τις αληθινές ετικέτες.
    :return: Λεξικό με τις μετρικές ακρίβειας (accuracy), F1-score, precision και recall.
    """
    logits, labels = eval_pred
    # Επιλογή της ετικέτας με τη μέγιστη τιμή πιθανότητας για κάθε δείγμα.
    predictions = logits.argmax(axis=-1)

    # Υπολογισμός των μετρικών precision, recall, f1 με χρήση του μέσου όρου "binary" (δυαδική ταξινόμηση).
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro")

    # Υπολογισμός της ακρίβειας.
    acc = accuracy_score(labels, predictions)

    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,                           # Το μοντέλο που θα εκπαιδευτεί.
    args=training_args,                    # Οι ορισμένες παράμετροι εκπαίδευσης.
    train_dataset=train_dataset,           # Το σύνολο δεδομένων εκπαίδευσης.
    eval_dataset=test_dataset,             # Το σύνολο δεδομένων αξιολόγησης (δοκιμής).
    compute_metrics=compute_metrics,       # Συνάρτηση υπολογισμού των μετρικών αξιολόγησης.
    callbacks=[early_stopping]             # Callback για το early stopping.
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.920500,0.861521,0.612917,0.611861,0.617234,0.612917
2,0.833400,0.808569,0.659861,0.656186,0.673518,0.659861
3,0.563900,0.853370,0.673750,0.672523,0.680795,0.673750


TrainOutput(global_step=675, training_loss=0.8152236097830313, metrics={'train_runtime': 3554.4378, 'train_samples_per_second': 1.519, 'train_steps_per_second': 0.19, 'total_flos': 1538455563878400.0, 'train_loss': 0.8152236097830313, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Πρόβλεψη στο test set χρησιμοποιώντας τον εκπαιδευμένο trainer.
predictions = trainer.predict(test_dataset)

# Εξαγωγή των αληθινών ετικετών (labels) από το test_dataset.
y_true = np.array([example['labels'] for example in test_dataset])

# Εξαγωγή των προβλεπόμενων ετικετών: επιλέγουμε την κλάση με τη μεγαλύτερη τιμή πιθανοτήτων για κάθε δείγμα.
y_pred = np.argmax(predictions.predictions, axis=1)


# Εκτύπωση του Classification Report για την αξιολόγηση της απόδοσης του μοντέλου.
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

Classification Report:
              precision    recall  f1-score   support

    Negative       0.64      0.77      0.70      2400
     Neutral       0.75      0.59      0.66      2400
    Positive       0.66      0.66      0.66      2400

    accuracy                           0.67      7200
   macro avg       0.68      0.67      0.67      7200
weighted avg       0.68      0.67      0.67      7200



## GPT

In [ ]:
import pandas as pd

balanced_tweets = pd.read_csv('/content/Balanced_Tweets.csv')

from datasets import Dataset
from sklearn.model_selection import train_test_split
SEED = 42

train_texts, test_texts, train_labels, test_labels = train_test_split(
    balanced_tweets['CleanTweet'], balanced_tweets['Sentiment'], test_size=0.8, random_state=SEED, stratify=balanced_tweets['Sentiment']
)

train_dataset = Dataset.from_dict({'CleanTweet': train_texts, 'Sentiment': train_labels})
test_dataset = Dataset.from_dict({'CleanTweet': test_texts, 'Sentiment': test_labels})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT models don't have pad_token by default → we set it manually
tokenizer.pad_token = tokenizer.eos_token

num_labels = 3

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    pad_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def tokenize_function(example):
    return tokenizer(
        example["CleanTweet"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.rename_column("Sentiment", "labels")
test_dataset = test_dataset.rename_column("Sentiment", "labels")

train_dataset.set_format("torch")
test_dataset.set_format("torch")

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./gpt-results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./gpt-logs",
    logging_steps=10,
    seed=42,
)

early_stopping = EarlyStoppingCallback(early_stopping_patience=2)

def compute_metrics(eval_pred):
    import numpy as np
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping]
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-5-fd1d6f53cee0>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: puaphantom (puaphantom-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.973700,0.975290,0.530556,0.524149,0.534589,0.530556
2,0.890800,0.895510,0.604028,0.596136,0.618054,0.604028
3,0.681600,0.875846,0.643194,0.641773,0.661277,0.643194
4,0.543800,0.842516,0.657222,0.654937,0.669860,0.657222
5,0.456200,0.891726,0.669028,0.668693,0.670974,0.669028
6,0.339700,1.098828,0.667639,0.667384,0.673461,0.667639


TrainOutput(global_step=1350, training_loss=0.7140201837045175, metrics={'train_runtime': 461.5965, 'train_samples_per_second': 38.995, 'train_steps_per_second': 4.874, 'total_flos': 352769723596800.0, 'train_loss': 0.7140201837045175, 'epoch': 6.0})

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

predictions = trainer.predict(test_dataset)
y_pred = predictions.predictions.argmax(axis=-1)
y_true = [example["labels"] for example in test_dataset]

print(classification_report(y_true, y_pred, target_names=['Negative', 'Neutral', 'Positive']))

              precision    recall  f1-score   support

    Negative       0.68      0.67      0.67      2400
     Neutral       0.60      0.77      0.67      2400
    Positive       0.73      0.54      0.62      2400

    accuracy                           0.66      7200
   macro avg       0.67      0.66      0.65      7200
weighted avg       0.67      0.66      0.65      7200

